**Training a movie review classification model to distinguish between positive and negative reviews**

In [ ]:
# Run this only when necessary
!git config --global user.name "nsahsimon"
!git config --global user.email "nsahsimonai01@gmail.com"
!git add .
!git commit -m "Cleaned text"
!git push origin main

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/deep_learning_projects/RNN_projects/text_classification

/content/drive/MyDrive/deep_learning_projects/RNN_projects/text_classification


In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np
import sklearn
import os
import requests
import shutil
import urllib.request
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dropout, Dense
import tensorflow.keras.datasets as datasets


**Download and organize data**

In [10]:
import os
import shutil
import requests

# Download the IMDB dataset
url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
filename = 'aclImdb_v1.tar.gz'
response = requests.get(url, stream=True)
with open(filename, 'wb') as f:
    f.write(response.content)

# Extract the dataset
shutil.unpack_archive(filename)

# Create the directory structure for the data
os.makedirs('data/train/pos')
os.makedirs('data/test/pos')
os.makedirs('data/train/neg')
os.makedirs('data/test/neg')

# Move the positive and negative reviews to the appropriate directories
for dataset in ['train', 'test']:
    for sentiment in ['pos', 'neg']:
        source_dir = os.path.join('aclImdb', dataset, sentiment)
        target_dir = os.path.join('data', dataset, sentiment)
        files = os.listdir(source_dir)
        for file in files:
            source_file = os.path.join(source_dir, file)
            target_file = os.path.join(target_dir, file)
            shutil.copyfile(source_file, target_file)

# Remove the extracted dataset
shutil.rmtree('aclImdb')

In [ ]:
x_train = []
y_train = []
x_test = []
y_test = [] 

for dataset in ['train', 'test']:
  for sentiment in ['pos', 'neg']:
    dir_path = os.path.join("data", dataset, sentiment)
    files = os.listdir(dir_path)
    for _file in files:
      file_path = os.path.join(dir_path, _file)
      df = pd.read_csv(file_path, sep='\t', names=['review'])
      if dataset == 'train':
        x_train.append(df['review'].values[0])
        if sentiment == 'pos':
          y_train.append(1)
        else :
          y_train.append(0)
      else:
        x_test.append(df['review'].values[0])
        if sentiment == 'pos':
          y_test.append(1)
        else :
          y_test.append(0)


# Convert to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [129]:
# Permute the train set
train_len = len(x_train)
train_permutation = np.random.permutation(train_len)
x_train = x_train[train_permutation]
y_train = y_train[train_permutation]

In [130]:
print(x_train[0])
print(y_train[0])

The Matador stars Pierce Brosnan as a burned out assassin. He's James Bond gone to seed, in too-tight, garish clothes, gold chains, and an ugly haircut. Our struggling assassin, Julian Noble, is in Mexico, trying to regain his nerve. Staying at the same hotel is a likable, down-on-his luck businessman Danny Wright (Greg Kinnear), also trying to regain his equilibrium. Danny is desperate to close a deal and return to his wife in Denver (Hope Davis) with good news.<br /><br />Noble and Wright unexpectedly become friends. Wright convinces Noble to reveal certain techniques, which he demonstrates at a bullfight. Noble is eventually targeted by his employers and shows up in Denver. <br /><br />Writer and director Richard Shepard did the Q&A after this delightful movie at the Austin Film Festival. Shepard was also down on his luck. After suffering the loss of his agent and rejection of recent scripts, he decided to write a story no one would buy and create a character no one would want to pl

In [108]:
print(f"y_train: {y_train[0]} , x_train: {x_train[0]}")

y_train: 1 , x_train: Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn't really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I'd have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.


In [109]:
print(f"y_test: {y_test[0]} , x_test: {x_test[0]}")

y_test: 1 , x_test: ["Previous reviewer Claudio Carvalho gave a much better recap of the film's plot details than I could. What I recall mostly is that it was just so beautiful, in every sense - emotionally, visually, editorially - just gorgeous.<br /><br />If you like movies that are wonderful to look at, and also have emotional content to which that beauty is relevant, I think you will be glad to have seen this extraordinary and unusual work of art.<br /><br />On a scale of 1 to 10, I'd give it about an 8.75. The only reason I shy away from 9 is that it is a mood piece. If you are in the mood for a really artistic, very romantic film, then it's a 10. I definitely think it's a must-see, but none of us can be in that mood all the time, so, overall, 8.75."]


In [ ]:
# display results

In [70]:
dir_path = "data/train/neg"
files = os.listdir(dir_path)
_file = files[0]
file_path = os.path.join(dir_path, _file)
try:
  df = pd.read_csv(file_path, header=None, sep='\t', names=['review'])
  print(df["review"].values)
except:
  print("Failed to read data frame")

combined_df.append(df)

['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, e

In [36]:
# merged_frames = pd.concat([combined_df[0], combined_df[0]], axis = 1)
print(combined_df[0])

[]


In [79]:
import numpy as np

# create two example arrays
x = np.array([1, 2, 3, 4, 5])
y = np.array([10, 20, 30, 40, 50])

# generate a random permutation of indices
perm = np.random.permutation(len(x))

# shuffle both arrays in the same way using the permutation
x_shuffled = x[perm]
y_shuffled = y[perm]

# print the shuffled arrays
print(x_shuffled)
print(y_shuffled)

[2 3 5 1 4]
[20 30 50 10 40]


**Loading the data**

In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import numpy as np



In [78]:
# Sample text data
text = "This is a,  bad bad boy. here -sample 1    text 2 string !   "
print(text)

This is a,  bad bad boy. here -sample 1    text 2 string !   


In [79]:
# Remove punctuation and convert to lowercase
text = text.translate(str.maketrans("", "", string.punctuation)).lower()
print(text)
print(type(text))

this is a  bad bad boy here sample 1    text 2 string    
<class 'str'>


In [80]:
# Remove digits 
text = text.translate(str.maketrans("", "",string.digits))

# Remove whitespaces
text = text.replace("  "," ")
print(text)
print(type(text))

this is a bad bad boy here sample   text string  
<class 'str'>


In [ ]:
split_text = text.split()

In [82]:
print(split_text)

['this', 'is', 'a', 'bad', 'bad', 'boy', 'here', 'sample', 'text', 'string']


In [83]:
texts = [split_text]  # add the other texts
texts.append("my world".split())

In [84]:
# Tokenize the text and remove stop words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print(word_index)

{'bad': 1, 'this': 2, 'is': 3, 'a': 4, 'boy': 5, 'here': 6, 'sample': 7, 'text': 8, 'string': 9, 'my': 10, 'world': 11}


In [85]:
sequences = tokenizer.texts_to_sequences(texts) # A sequences are justs texts where the actual value has been replaced with the word_index
print(sequences)
print(type(sequences))

[[2, 3, 4, 1, 1, 5, 6, 7, 8, 9], [10, 11]]
<class 'list'>


In [86]:
stop_words = set(['the', 'and', 'or', 'a', 'an', 'of', 'in', 'to', 'at', 'with'])

In [87]:
filtered_texts = []
for text in texts:
    filtered_texts.append([word for word in text if word not in stop_words])

In [88]:
filtered_sequences = tokenizer.texts_to_sequences(filtered_texts)
print(filtered_sequences)

[[2, 3, 1, 1, 5, 6, 7, 8, 9], [10, 11]]


In [90]:
# Padding the sequences to make them of equal length
padded_sequences = pad_sequences(filtered_sequences, maxlen=10, padding='post')

print(padded_sequences)

[[ 2  3  1  1  5  6  7  8  9  0]
 [10 11  0  0  0  0  0  0  0  0]]


In [92]:
x_train = np.array(padded_sequences)
print(x_train)
print(type(x_train))

[[ 2  3  1  1  5  6  7  8  9  0]
 [10 11  0  0  0  0  0  0  0  0]]
<class 'numpy.ndarray'>
